In [ ]:
from absl import logging

import tensorflow as tf
import re

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)


In [ ]:
from glob import glob 

filenames = glob('../my-markdown-notes/*.md')
filenames.remove('../my-markdown-notes/README.md')
filenames

# load docs into memory
docs = []
for filename in filenames:
    f = open(filename, 'r')
    docs.append(f.read())
    
clean_filenames = [name.split('/')[2].replace('.md','') for name in filenames]
clean_filenames

In [ ]:
def preprocess(doc):
    """
    Remove urls, linebreaks, dashes, equations, html, etc. 
    
    Most of it is done through regular expressions. 
    """
    new_doc = doc.replace('\n',' ')
    
    operations = [r'\$\$(.+?)\$\$',
                  r'\$(.+?)\$',
                  r'\((.+?)\)',
                 'r\[(.+?)\]',
                 r'\#',
                 r'---(.+?)---',
                 r'[\*\-\_\\]',
                 r'<[^>]*>']
    
    
    for op in operations:
        new_doc = re.sub(op, '', new_doc)

    return new_doc

new_docs = [preprocess(doc) for doc in docs]
message_embeddings = embed(new_docs)

In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  plt.figure(figsize=(10,10))
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_, labels):
  message_embeddings_ = embed(messages_)
  plot_similarity(labels, message_embeddings_, 90)


In [ ]:
run_and_plot(new_docs, clean_filenames)

In [ ]:
class DocSim:
    def __init__(self,message_embeddings):
        corr = np.inner(message_embeddings, message_embeddings)
        np.fill_diagonal(corr,0) # fill diagonal
        self.corr = corr
        
    def rdi(self, index):
        """
        Returns correlated docs
        """
        return self.corr[index,:].argsort()[::-1][0:3]
    
    def build_graph(self, labels):
        """
        labels for the docs; for example, you can use the 
        filenames. Should be in the same order as the message embeddings. 
        """
        self.graph = {}
        for i,name in enumerate(labels):
            self.graph[name] = [labels[key] for key in self.rdi(i)]
        
        

In [ ]:
ds = DocSim(message_embeddings)
ds.build_graph(filenames)
ds.graph


In [ ]:
for i,_ in enumerate(clean_filenames):
    print(clean_filenames[i])
    j = np.argmax(corr[i,:])
    print(clean_filenames[j])
    print(corr[i,j])
